In [4]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn
from pathlib import Path
import numpy as np

In [5]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

In [6]:
dataset = pd.read_csv("../excel_files/dataset.csv")

In [7]:
X = dataset.drop(columns=['Survived', 'PassengerId'])
y = dataset['Survived']

In [8]:
X_tensor = torch.tensor(X.values, dtype=torch.float).to(device)
y_tensor = torch.tensor(y.values, dtype=torch.float).to(device)

X_tensor, y_tensor, X_tensor.shape, y_tensor.shape, X_tensor.device, y_tensor.device

(tensor([[3.0000, 0.0000, 0.2570,  ..., 0.0142, 0.0000, 0.0000],
         [1.0000, 1.0000, 0.4453,  ..., 0.1391, 1.0000, 0.0000],
         [3.0000, 1.0000, 0.3041,  ..., 0.0155, 0.0000, 1.0000],
         ...,
         [3.0000, 1.0000, 0.3450,  ..., 0.0458, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.3041,  ..., 0.0586, 1.0000, 1.0000],
         [3.0000, 0.0000, 0.3747,  ..., 0.0151, 2.0000, 1.0000]],
        device='mps:0'),
 tensor([0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
         0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
         1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
   

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

len(X_train), len(y_train), len(X_test), len(y_test), X_train.shape, y_train.shape, X_test.shape, y_test.shape

(712,
 712,
 179,
 179,
 torch.Size([712, 8]),
 torch.Size([712]),
 torch.Size([179, 8]),
 torch.Size([179]))

In [10]:
class TitanicModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(in_features=X_train.shape[1], out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=len(y_train.shape))
        )
    def forward(self, x):
        return self.layer_stack(x)

In [11]:
torch.manual_seed(42)

model_0 = TitanicModelV0()
model_0.to(device)

model_0.state_dict(), model_0

(OrderedDict([('layer_stack.0.weight',
               tensor([[ 0.2703,  0.2935, -0.0828,  0.3248, -0.0775,  0.0713, -0.1721,  0.2076],
                       [ 0.3117, -0.2594,  0.3073,  0.0662,  0.2612,  0.0479,  0.1705, -0.0499],
                       [ 0.2725,  0.0523, -0.1651,  0.0901, -0.1629, -0.0415, -0.1436,  0.2345],
                       [-0.2791, -0.1630, -0.0998, -0.2126,  0.0334, -0.3492,  0.3193, -0.3003],
                       [ 0.2730,  0.0588, -0.1148,  0.2185,  0.0551,  0.2857,  0.0387, -0.1115],
                       [ 0.0950, -0.0959,  0.1488,  0.3157,  0.2044, -0.1546,  0.2041,  0.0633],
                       [ 0.1795, -0.2155, -0.3500, -0.1366, -0.2712,  0.2901,  0.1018,  0.1464],
                       [ 0.1118, -0.0062,  0.2767, -0.2512,  0.0223, -0.2413,  0.1090, -0.1218],
                       [ 0.1083, -0.0737,  0.2932, -0.2096, -0.2109, -0.2109,  0.3180,  0.1178],
                       [ 0.3402, -0.2918, -0.3507, -0.2766, -0.2378,  0.1432,  0.1266,  

In [12]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(params=model_0.parameters(), lr=0.001)

In [13]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [14]:
torch.manual_seed(42)
epochs = 9000

for epoch in range(epochs):
    model_0.train()
    y_logits = model_0(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))  # Prediction thresholding for accuracy calculation
    loss = loss_fn(y_logits, y_train)
    acc = accuracy_fn(y_true=y_train, y_pred=y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model_0.eval()
    with torch.inference_mode():
        test_logits = model_0(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test, y_pred=test_pred)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch: [{epoch + 1}/{epochs}] | {loss = :.4f} | {acc = :.2f}% | {test_loss = :.4f} | {test_acc = :.2f}%")

Epoch: [1000/9000] | loss = 0.3396 | acc = 85.53% | test_loss = 0.5073 | test_acc = 79.89%
Epoch: [2000/9000] | loss = 0.2939 | acc = 87.64% | test_loss = 0.5684 | test_acc = 82.12%
Epoch: [3000/9000] | loss = 0.2775 | acc = 88.48% | test_loss = 0.6834 | test_acc = 83.24%
Epoch: [4000/9000] | loss = 0.2665 | acc = 88.76% | test_loss = 0.7996 | test_acc = 87.15%
Epoch: [5000/9000] | loss = 0.2594 | acc = 89.33% | test_loss = 0.8949 | test_acc = 86.03%
Epoch: [6000/9000] | loss = 0.2543 | acc = 89.19% | test_loss = 0.9974 | test_acc = 84.92%
Epoch: [7000/9000] | loss = 0.2502 | acc = 89.61% | test_loss = 1.0955 | test_acc = 84.36%
Epoch: [8000/9000] | loss = 0.2471 | acc = 89.33% | test_loss = 1.1979 | test_acc = 84.36%
Epoch: [9000/9000] | loss = 0.2447 | acc = 89.89% | test_loss = 1.3060 | test_acc = 84.92%


In [15]:
MODEL_NAME = "titanic_model_0.pth"
MODEL_PATH = Path("../models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
torch.save(obj=model_0.state_dict(),
           f=MODEL_SAVE_PATH)

In [16]:
result_dataset = pd.read_csv("../excel_files/result.csv")

In [17]:
X_result = result_dataset.drop(columns=['PassengerId'])

In [18]:
X_result_tensor = torch.tensor(X_result.values, dtype=torch.float).to(device)

In [19]:
model_0.eval()
with torch.inference_mode():
    y_result_logits = model_0(X_result_tensor)

y_result_labels = torch.round(torch.sigmoid(y_result_logits)) 

In [20]:
y_result_labels = y_result_labels.cpu().numpy().astype(int)

In [21]:
result_dataset['Survived'] = y_result_labels

In [22]:
result_dataset.to_csv("../excel_files/final_results.csv", index=False)